In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import torch_geometric
import networkx as nx
import os
import torch
import pandas as pd

from tqdm import tqdm
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from models.gat_transformer import GATConvTransformer, GATConvLSTM
from sklearn.preprocessing import LabelEncoder
from pathlib import Path
from torch_geometric.data import Batch
from models.gcn import GCNConvLSTM

from pathlib import Path

In [ ]:
def get_edge_index(node_num=12):
    src_nodes = [idx for idx in range(node_num) for _ in range(node_num)]
    tgt_nodes = [idx for _ in range(node_num) for idx in range(node_num)]
    edge_index = torch.tensor(np.array([src_nodes, tgt_nodes]), dtype=torch.long)
    return edge_index

In [ ]:
diagnostics_df = pd.read_excel('data/Diagnostics.xlsx')

In [ ]:
le = LabelEncoder()
le.fit(diagnostics_df.Rhythm.unique())

In [ ]:
le.classes_

In [ ]:
diagnostics_df['label'] = le.transform(diagnostics_df.Rhythm.values)

In [ ]:
diagnostics_df['label']

In [ ]:
diagnostics_df['label'].value_counts()

In [ ]:
data_list = []
data_path = Path('data', 'ECGDataDenoised')
for file_name in tqdm(data_path.iterdir()):
    if file_name.suffix == '.csv':
        sub_name = file_name.stem
        y = torch.tensor(diagnostics_df.loc[diagnostics_df.FileName == sub_name].label.values[0])
        df = pd.read_csv(file_name, header=None)
        if len(df) != 5000:
            print(file_name)
            continue
        x = torch.Tensor(np.array([df[col].values for col in df.columns]))
        edge_index = get_edge_index(node_num=12)
        data = Data(x=x, edge_index=edge_index, y=y)
        data_list.append(data)

In [ ]:
train_loader = DataLoader(data_list[:8000], batch_size=8, shuffle=True)
test_loader = DataLoader(data_list[8000:], batch_size=8)

In [ ]:
instance = next(iter(train_loader))

In [ ]:
instance

In [ ]:
#g = torch_geometric.utils.to_networkx(instance, to_undirected=True)
#nx.draw(g)

In [ ]:
instance.y[0]

In [ ]:
hidden_channels = 250
device = 'cpu' if not torch.cuda.is_available() else 'cuda'

In [ ]:
# Training Graph Neural Networks for Graph Classification
# Embed each node by performing multiple rounds of message passing
# Aggregate node embeddings into a unified graph embedding (readout layer)
# Train a final classifier on the graph embedding
 
# There exists multiple readout layers in literature, but the most common one is to simply take the average of node embeddings

# For ex: In GCNConv we use the  ReLU(𝑥)=max(𝑥,0)  activation for obtaining localized node embeddings,
# before we apply our final classifier on top of a graph readout layer.

# PyTorch Geometric provides this functionality via torch_geometric.nn.global_mean_pool, 
# which takes in the node embeddings of all nodes in the mini-batch and the assignment vector batch 
# to compute a graph embedding of size [batch_size, hidden_channels] for each graph in the batch.

# The final architecture for applying GNNs to the task of graph classification then looks as follows and allows for complete end-to-end training:

from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.nn import GraphConv


class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GNN, self).__init__()
        self.conv1 = GraphConv(-1, hidden_channels)
        self.conv2 = GraphConv(hidden_channels, hidden_channels)
        self.conv3 = GraphConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, len(le.classes_))

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        x = global_mean_pool(x, batch)

        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(-1, hidden_channels)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, len(le.classes_))

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x


class GAT(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GAT, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GATConv(-1, hidden_channels)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv2 = GATConv(hidden_channels, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv3 = GATConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, len(le.classes_))

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x
    
class GCNLSTM(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCNLSTM, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConvLSTM(1, hidden_channels)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, len(le.classes_))

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x
       
# Define the models
model1 = GCN(hidden_channels=hidden_channels).to(device)
model2= GAT(hidden_channels=hidden_channels).to(device)
model3 = GNN(hidden_channels=hidden_channels).to(device)
model4 = GCNLSTM(hidden_channels=hidden_channels).to(device)

# Print them 
print(model1)
print(model2)
print(model3)
print(model4)

In [ ]:
# Set model paramters and model type
def set_model_parameters(model_type, lr=0.01):
    model = model_type
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = torch.nn.CrossEntropyLoss().to(device)
    return model, optimizer, criterion

# Train the model
def train(model, optimizer,criterion):
    model.train()

    for idx, data in enumerate(train_loader):  # Iterate in batches over the training dataset.
        data.x = data.x.to(device)
        data.y = data.y.to(device)
        data.edge_index = data.edge_index.to(device)
        data.batch = data.batch.to(device)
        out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
        
        if idx % 500 == 0:
            print(f'{idx}tr) loss: {loss}')
    return

# Test the model 
def test(loader, model):
    model.eval()

    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        data.x = data.x.to(device)
        data.y = data.y.to(device)
        data.edge_index = data.edge_index.to(device)
        data.batch = data.batch.to(device)
        out = model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.

# Training and Testing Pipeline 
def running_epochs(model,optimizer,criterion):
    for epoch in range(1, 10):
        train(model,optimizer,criterion)
        train_acc = test(train_loader, model)
        test_acc = test(test_loader, model)
        print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

In [ ]:
# Experiment 1: GCN Baseline 
model, optimizer, criterion = set_model_parameters(model1, lr=0.001)
running_epochs(model,optimizer,criterion)

In [ ]:
# Experiment 4: GCN Baseline 
model, optimizer, criterion = set_model_parameters(model4, lr=0.001)
running_epochs(model,optimizer,criterion)